In [0]:
from google.colab import auth
auth.authenticate_user()


from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Flatten,Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
np.random.seed(3)

In [0]:
np.random.seed(3)
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/trainingSet',
                                                 target_size = (560, 420),
                                                 batch_size = 5,                                                
                                                 class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/testSet',
                                            target_size = (560, 420),
                                            batch_size = 5,                                            
                                            class_mode = 'categorical')

Found 120 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [0]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(2, 2),
                 activation='relu',input_shape=(560,420,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(10, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(1, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax' ))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=2e-5),metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 559, 419, 28)      364       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 279, 209, 28)      0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 279, 209, 28)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 279, 209, 10)      290       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 139, 104, 10)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 139, 104, 1)       11        
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 69, 52, 1)        

In [0]:
early_stopping_callback = EarlyStopping(monitor='val_loss',mode='min', patience=5)
checkpoint_callback = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit_generator(
        train_generator,
        steps_per_epoch=24,
        epochs=10,
        validation_data=test_generator,
        validation_steps=16,
        callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/10
24/24 [==============================] - 12s 513ms/step - loss: 0.7012 - acc: 0.4083 - val_loss: 0.6927 - val_acc: 0.6875

Epoch 00001: val_loss improved from inf to 0.69265, saving model to best_model_cnn.h5
Epoch 2/10
24/24 [==============================] - 11s 473ms/step - loss: 0.6899 - acc: 0.5500 - val_loss: 0.6925 - val_acc: 0.7375

Epoch 00002: val_loss improved from 0.69265 to 0.69253, saving model to best_model_cnn.h5
Epoch 3/10
24/24 [==============================] - 11s 478ms/step - loss: 0.6947 - acc: 0.4583 - val_loss: 0.6926 - val_acc: 0.7625

Epoch 00003: val_loss did not improve from 0.69253
Epoch 4/10
24/24 [==============================] - 12s 479ms/step - loss: 0.6884 - acc: 0.5250 - val_loss: 0.6925 - val_acc: 0.7625

Epoch 00004: val_loss improved from 0.69253 to 0.69250, saving model to best_model_cnn.h5
Epoch 5/10
24/24 [==============================] - 12s 481ms/step - loss: 0.6817 - acc: 0.5583 - val_loss: 0.6924 - val_acc: 0.8000

Epoch 00005: 

In [0]:
print("--TrianSet_Evaluate--")
scores=model.evaluate_generator(train_generator,steps=24)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TrianSet_Evaluate--
acc: 80.00%


In [0]:
print("--TestSet_Evaluate--")

scores=model.evaluate_generator(test_generator,steps=16)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TestSet_Evaluate--
acc: 81.25%
